# İmport library

In [ ]:

import numpy as np 
import pandas as pd 
import seaborn as sns
import os, cv2, random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization


In [ ]:
import os
print(os.listdir("../input"))

In [ ]:
import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

In [ ]:
train_image_list=os.listdir("train")
test_image_list=os.listdir("test1")

In [ ]:
image_width=128
image_height=128
image_channels=3
image_size=(image_width,image_height)

In [ ]:
filenames = train_image_list
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'cat':
        categories.append(0)
    else:
        categories.append(1)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df.head()

In [ ]:
label=[]
for img in train_image_list:
  label.append(img.split(".")[0])
sns.countplot(label,palette="pastel")

plt.title("cats and dogs")

# sample image

In [ ]:
n =5
fig, axes = plt.subplots(1, n, figsize=(20,20))
for i in range(n):
    directory=random.choice(os.listdir("./train/"))
    img=cv2.imread("./train/"+directory)
    axes[i].imshow(img)
    axes[i].axis("off")
plt.show()

# Build Model

In [ ]:
model=Sequential()

#convolutional layer

model.add(Conv2D(32,(3,3),activation="relu",input_shape=(image_width,image_height,image_channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#convolutional layer
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

#Flattening
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(BatchNormalization())

model.add(Dense(2,activation="softmax"))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
df["category"]=df["category"].replace({0:"cat",
                                      1:"dog"})

In [ ]:
train_df,validate_df=train_test_split(df,test_size=0.3,random_state=42)
train_df=train_df.reset_index(drop=True)
validate_df=validate_df.reset_index(drop=True)

In [ ]:
total_train=train_df.shape[0]
total_validate=train_df.shape[0]
batch_size=32

In [ ]:
train_datagen=ImageDataGenerator(
rotation_range=15,
rescale=1./255,
shear_range=0.1,
zoom_range=0.2,
horizontal_flip=True,
width_shift_range=0.1,
height_shift_range=0.1)

train_generator=train_datagen.flow_from_dataframe(
train_df,
"train",
x_col="filename",
y_col="category",
target_size=image_size,
class_mode="categorical",
batch_size=batch_size)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "train", 
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
img_sample=random.choice(os.listdir("train"))
sample=cv2.imread("train"+img_sample)

example_df = train_df.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    "train", 
    x_col='filename',
    y_col='category',
    target_size=image_size,
    class_mode='categorical'
)

# sample image

In [ ]:
plt.figure(figsize=(15,15))
n=9
for i in range(n):
    plt.subplot(3,3,i+1)
    for X_batch, Y_batch in example_generator:
        image=X_batch[0]
        plt.imshow(image)
        plt.axis("off")
        break
        
plt.tight_layout()
plt.show()

# Fit the model

In [ ]:
epochs=50
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size
)

In [ ]:
plt.plot(history.history["accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["Train"],loc="upper left")